In [1]:
import sys

In [2]:
import os

In [3]:
from pathlib import Path

In [4]:
project_dir = Path(__name__).resolve().parents[1]

In [5]:
sys.path.insert(0, str(project_dir))

In [6]:
sys.path.insert(0, str(project_dir/'src'/'data'))

In [7]:
from scrape_bol import *

In [8]:
from searchterm import *

In [9]:
from datetime import datetime

In [10]:
s = SearchTerm(search_term='philips', search_domain = 'bcc.nl', max_pages = 1,
              driver_method = 'requests')

In [27]:
s = SearchTerm(search_term='illy', search_domain = 'bol.com', max_pages = 1)

In [ ]:
len(s.productlist)

In [ ]:
print(s.Scraper.page)

In [ ]:
dir(s)

In [ ]:
datetime.now().strftime("%Y-%m-%d %I:%M%p")

In [ ]:
to_save_data = ['\t'.join([p.provider if p.provider else '', 
                          p.provider_id if p.provider_id else '', 
                           p.name  if p.name else '', 
                           p.price  if p.price else '', 
                           p.original_price  if p.original_price else '' ]) 
                for p in s.productlist]

In [ ]:
with open("../data/raw/price_tracker.tsv", 'a') as file:
    file.writelines(to_save_data)

In [10]:
bol = BolScraper('https://www.bol.com/nl/nl/s/?searchtext=brita')

In [11]:
pl_bs4 = bol.page.find_all('li', attrs = {'class': 'product-item--row'})

In [12]:
example = [p for p in pl_bs4 if p.attrs['data-id'] == '9200000074016762'][0]

In [237]:
example.find('span', attrs = {'data-test': 'price'}).contents[0].strip()

'42'

In [238]:
example.find('sup', attrs = {'data-test': 'price-fraction'}).contents[0].strip()

'94'

In [14]:
pl = bol.get_productlist()

In [15]:
pl

[]

In [16]:
bol = BolScraper('https://www.bol.com/nl/nl/s/?page=8&searchtext=philips+sonicare&view=list') # No products

In [17]:
pl = bol.get_productlist()

In [21]:
bol._productlist_bs4

[]

In [22]:
bol.locators

{'product_search_result_locator': ('li',
  {'attrs': {'class': 'product-item--row'}}),
 'provider_id_locator': None,
 'name_locator': None,
 'url_locator': None,
 'price_locator': None,
 'original_price_locator': None}

In [18]:
pl

[]

In [ ]:
# pl

In [ ]:
from pathlib import Path

In [ ]:
project_dir = Path(__name__).resolve().parents[2]

In [ ]:
project_dir/'data'/'raw'/'example.csv'

# Create DB

In [ ]:
import sqlite3
from sqlite3 import Error

In [ ]:
conn = sqlite3.connect("../data/raw/price_tracker.db",  timeout=10)

In [ ]:
sql_create_products_table = """ CREATE TABLE IF NOT EXISTS projects (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    begin_date text,
                                    end_date text
                                ); """

sql_create_prices_table = """CREATE TABLE IF NOT EXISTS prices (
                                datetime datetime NOT NULL,
                                provider text ,
                                provider_id text,
                                name text,
                                price numeric,
                                original_price numeric
                            );"""

In [ ]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [ ]:
create_table(conn, sql_create_prices_table)

In [ ]:
conn.close()

# Pandas

In [14]:
import pandas as pd

data_dir = project_dir/'data'/'external'
new_file = data_dir /'new_price.tsv'
old_file = data_dir / 'old_price.tsv'

df_new = pd.read_csv(new_file, sep = '\t')
df_old = pd.read_csv(old_file, sep = '\t')
for df in [df_new, df_old]:
    df['provider_id'] = df['provider_id'].astype('str')

In [15]:
df = pd.merge(df_new, df_old, on = ['provider', 'provider_id'], 
        suffixes = (None, '_old'))

df['discount'] = 1 - df['price'] / df['price_old']

df_notify = df[df['discount'] >=0.1]


In [19]:
df_new.tail()

,provider,provider_id,name,price,original_price,hidden_price,URL
450,bcc,000000000000303147,Samsung Crystal UHD TV 43AU8070 (2021),649.0,NaN,NaN,https://bcc.nl/televisie/led-tv/4k-led-tv/sams...
451,bcc,000000000000285159,Philips LED Full HD TV 32PFS6805/12,329.0,NaN,NaN,https://bcc.nl/televisie/led-tv/hd-led-tv/phil...
452,bcc,000000000000308686,LG QHD monitor 32QN600-B.AEU,279.0,NaN,NaN,https://bcc.nl/computer/monitor/monitor-standa...
453,bcc,000000000000303153,Samsung QLED TV 32Q50A (2021),599.0,NaN,NaN,https://bcc.nl/televisie/qled-tv/4k-qled-tv/sa...
454,bcc,000000000000303647,Sony Bravia 4K Ultra HD TV KD55X81JAEP (2021),899.0,NaN,NaN,https://bcc.nl/televisie/led-tv/4k-led-tv/sony...


In [16]:
df_notify = df[df['discount'] != 0]

In [17]:
df_notify

,provider,provider_id,name,price,original_price,hidden_price,URL,name_old,price_old,original_price_old,hidden_price_old,URL_old,discount
20,bol,9200000092109413,"Philips 4300 series Sonische, elektrische tand...",60.70,NaN,60.70,https://bol.com/nl/nl/p/philips-4300-series-so...,"Philips 4300 series Sonische, elektrische tand...",74.99,NaN,74.99,https://bol.com/nl/nl/p/philips-4300-series-so...,0.190559
34,bol,9200000073431955,BRITA Maxtra+ Filterpatronen - 3 stuks - Wit,23.59,NaN,23.59,https://bol.com/nl/nl/p/brita-maxtra-filterpat...,BRITA Maxtra+ Filterpatronen - 3 stuks - Wit,23.25,NaN,23.25,https://bol.com/nl/nl/p/brita-maxtra-filterpat...,-0.014624
98,bol,9200000081014943,Dettol Magic Foam Navulling Vanille - Voordeel...,17.46,NaN,17.46,https://bol.com/nl/nl/p/dettol-magic-foam-navu...,Dettol Magic Foam Navulling Vanille - Voordeel...,17.64,NaN,17.64,https://bol.com/nl/nl/p/dettol-magic-foam-navu...,0.010204


In [17]:
df[df['name'].str.contains('Little Dutch Memorybox')]

,provider,provider_id,name,price,original_price,hidden_price,URL,name_old,price_old,original_price_old,hidden_price_old,URL_old,discount
77,bol,9200000125522811,Little Dutch Memorybox - Baby Geschenkset,23.69,NaN,23.69,https://bol.com/nl/nl/p/little-dutch-memorybox...,Little Dutch Memorybox - Baby Geschenkset,23.69,NaN,23.69,https://bol.com/nl/nl/p/little-dutch-memorybox...,0.0


In [23]:
df_notify['URL'].values[-1]

'https://bol.com/nl/nl/p/illy-iperespresso-koffie-home-cafeinevrij-6-x-18-capsules/9300000007109869/'

In [262]:
record = records[0]

In [263]:
records = df_notify.to_dict(orient = 'records')
r = []
for record in records:
    price = record['price']
    original_price = record['price_old']
    url = record['URL']

    r.append('\n'.join([record['provider'], record['name'], f'{price}~{original_price}~',
                            f'<{url}|Product Link>'
                           ])
            )

In [264]:
'\n\n'.join(r)

''

# Dotenv

In [ ]:
from dotenv import find_dotenv, load_dotenv

In [ ]:
load_dotenv(find_dotenv())

In [ ]:
f = project_dir.parent / 'Utils'

In [ ]:
f

# Selenium Driver

In [14]:
bol = BolScraper('https://www.bol.com/nl/nl/s/?searchtext=brita', driver_method = 'selenium')

In [15]:
pl = bol.get_productlist()

In [20]:
bol.driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="366ffb8fb0c52d2d66a149f2cd27670c")>

In [17]:
# pl

In [18]:
bol.close()

# MediaMarkt

In [27]:
from scrape_mediamarkt import *

In [28]:
url =  'https://www.mediamarkt.nl/nl/search.html?query=galaxy+tab+s7&searchProfile=onlineshop&channel=mmnlnl'

In [29]:
s = MediaMarktScraper(url, driver_method = 'selenium')

In [30]:
product_lists = s.page.find_all('div', attrs = {'class':'product-wrapper'})

In [106]:
product = product_lists[0]

In [126]:
prices = product.find('div', attrs = {'class': 'price', 'class': 'small'})

In [127]:
''.join([i.contents[0] for i in prices.contents]).strip().strip('-').replace(',', '.')

'849.'

In [128]:
old_prices = product.find('div', attrs = {'class': 'price-old', 'class': 'price'})

In [129]:
''.join([i.contents[0] for i in old_prices.contents]).strip().strip('-').replace(',', '.')

'979.'

In [65]:
'https://mediamarkt.nl'+ product.find('a').attrs['data-clickable-href']

'https://mediamarkt.nl/nl/product/_philips-sonicare-plaquedefense-hx6231-01-1371881.html'

In [66]:
product.attrs

{'class': ['product-wrapper'],
 'data-bundle': 'false',
 'data-modelnumber': '1371881',
 'data-age-rating': '0',
 'data-gtm-impression-event': 'EEC_PRODUCT_IMPRESSION',
 'data-gtm-impression-event-ext': 'product1371881',
 'data-reco-pid': 'MMNL1371881'}

In [79]:
product.find('img').attrs['alt']

'PHILIPS Sonicare PlaqueDefense HX6231/01'

In [16]:
s.close()

In [32]:
from scrape_bcc import *

In [33]:
s = BCCScraper('https://www.bcc.nl/search?fh_location=%2F%2Fcatalog01%2Fnl_NL%2Fchannel%3E%7Bm2ebcc2enl%7D&search=desktop+amd')

In [42]:
pl = s.page.find_all(**{'name':'div', 'attrs' : {'class':'lister-product'}})

In [43]:
product_bs4 = pl[0]

In [47]:
product_bs4.find('div', attrs = {'class': 'priceblock__price'})

In [51]:
name = product_bs4.find('h3', attrs = {'class': 'h2'}).text

In [52]:
name

'MSI gaming desktop MAG META 5 3SI-442MYS'

In [60]:
import json

In [61]:
json.loads(product_bs4.find('script', attrs = {'class': 'js-analytics-data'}).text)

{'name': 'MSI gaming desktop MAG META 5 3SI-442MYS',
 'id': '000000000000307902',
 'price': '1049.00',
 'brand': 'MSI',
 'category': 'computer/desktop/gaming-pc',
 'position': '1',
 'dimension12': 'V',
 'dimension43': 'Online op voorraad'}

In [65]:
product_bs4.find('a', attrs = {'class': 'lister-product__titlelink'}).attrs['href']

'/computer/desktop/gaming-pc/msi-gaming-desktop-mag-meta-5-3si-442mys/307902'

In [10]:
s = SearchTerm('philips', 'bcc.nl', max_pages = 2)

In [11]:
pl = s.productlist

In [13]:
len(pl)

36